In [3]:
import sys
!pip install skyfield

from skyfield.api import load
from skyfield.api import Topos

planets = load('de421.bsp')
earth, mars = planets['earth'], planets['mars']

ts = load.timescale()
t = ts.now()
astrometric = earth.at(t).observe(mars)
ra, dec, distance = astrometric.radec()

print(ra)
print(dec)
print(distance)

     |████████████████████████████████| 378kB 2.0MB/s eta 0:00:01
     |████████████████████████████████| 40kB 9.3MB/s  eta 0:00:01
     |████████████████████████████████| 174kB 8.7MB/s eta 0:00:01
  Created wheel for skyfield: filename=skyfield-1.31-cp37-none-any.whl size=409019 sha256=9b88211968230ab2e0fa2ff943390847ab9f4e264ec9e933f3f94142a218bcb1
  Stored in directory: /Users/jinyoungbae/Library/Caches/pip/wheels/9e/72/a8/6cd062de5215beb596061c26e6af2948277a9ece5f8f92e5f5
  Created wheel for jplephem: filename=jplephem-2.15-cp37-none-any.whl size=46020 sha256=8e43d2fe5634a5a53f02359584c42e9bc0b988fce3b351397d663d087d608bf5
  Stored in directory: /Users/jinyoungbae/Library/Caches/pip/wheels/01/71/9c/beaa33beb607ef67b62765324a10cd3584e30fbeef8856b7a8
Successfully built skyfield jplephem


[#################################] 100% de421.bsp


01h 08m 24.29s
+04deg 49' 42.0"
0.441688 au


In [4]:
boston = earth + Topos('42.3583 N', '71.0636 W')
astrometric = boston.at(t).observe(mars)
alt, az, d = astrometric.apparent().altaz()

print(alt)
print(az)

-19deg 22' 51.4"
63deg 37' 46.8"
